In [42]:
#importing libraries
import pandas as pd
from text_analytics.config import RAW_DATA_PATH

In [43]:
df_raw = pd.read_csv(RAW_DATA_PATH, index_col=0)
df_raw.head()

,sentiment
review,
"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br

In [44]:
#importing libraries
from rouge import Rouge
import numpy as np
import numpy.typing as npt
from typing import List, Any
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict

class ExtractiveTextSummarizer:
    def __init__(self, article: str) -> None:
        self.article = article
        self.frequency_table = defaultdict(int)
        

    def _create_dictionary_table(self, stemmer: Any = None) -> dict:
   
        #removing stop words
        stop_words = set(stopwords.words("english"))
        word_vector = word_tokenize(self.article)

        # instantiate the stemmer 
        if stemmer is None: 
            stemmer = PorterStemmer()

        stemmed_word_vector = [stemmer.stem(word) for word in word_vector]
        for word in stemmed_word_vector:
            if word not in stop_words:
                self.frequency_table[word] += 1

        return self.frequency_table


    def _calculate_sentence_scores(self, sentences: npt.ArrayLike) -> dict:   

        #algorithm for scoring a sentence by its words
        sentence_weights = defaultdict(int)

        for sentence in sentences:
            sentence_wordcount_without_stop_words = 0

            for word_weight in self.frequency_table:
                sentence_weights[sentence[:7]] += self.frequency_table[word_weight]

                if word_weight in sentence.lower():
                    sentence_wordcount_without_stop_words += 1

            sentence_weights[sentence[:7]] /= sentence_wordcount_without_stop_words

        return sentence_weights


    def _calculate_threshold_score(self, sentence_weight: dict) -> float:
        return np.mean(list(sentence_weight.values()))


    def _get_article_summary(self, sentences: npt.ArrayLike, sentence_weights: dict, threshold: float) -> str:
        article_summary = [sentence for sentence in sentences if sentence[:7] in sentence_weights and sentence_weights.get(sentence[:7]) >= threshold]

        return " ".join(article_summary)

    def run_article_summary(self):

        #creating a dictionary for the word frequency table
        _ = self._create_dictionary_table()

        #tokenizing the sentences
        sentences = sent_tokenize(self.article)

        #algorithm for scoring a sentence by its words
        sentence_scores = self._calculate_sentence_scores(sentences)

        # getting the threshold
        threshold = self._calculate_threshold_score(sentence_scores)

        #producing the summary
        article_summary = self._get_article_summary(sentences, sentence_scores, 0.95 * threshold)

        return article_summary

    def get_rouge_score(self, hypothesis_text: str, reference_text: str) -> npt.ArrayLike:
        rouge = Rouge()
        scores = rouge.get_scores(hypothesis_text, reference_text)
        return scores


if __name__ == '__main__':
    article = df_raw.sample(1).index[0]
    print(f"Original Review: \n{article}")
    print('-' * 200)
    extractive_summarizer = ExtractiveTextSummarizer(article=article)
    result = extractive_summarizer.run_article_summary()
    print(f"Summarised Review: \n{result}")
    print('-' * 200)
    # this line is POC for now since we don't have the reference text
    print(extractive_summarizer.get_rouge_score(hypothesis_text=result, reference_text=result))

Original Review: 
This movie was terrible. The plot was terrible and unbelievable. I cannot recommend this movie. Where did this movie come from? This movie was not funny and wasted the talent of some great actors and actresses including: Gary Sinise, Kathy Bates, Joey Lauren Adams, and Jennifer Tilly.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Summarised Review: 
The plot was terrible and unbelievable. I cannot recommend this movie. Where did this movie come from?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]
